In [2]:
import requests
import json
import pandas as pd
import numpy as np
import difflib
import regex as re

In [3]:
# Fetch item mapping (name → id, etc.)
def fetch_item_mapping():
    url = "https://prices.runescape.wiki/api/v1/osrs/mapping"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return []

In [4]:
# Fetch latest GE prices
def fetch_ge_prices():
    url = "https://prices.runescape.wiki/api/v1/osrs/latest"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return []

In [5]:
item_map = fetch_item_mapping()
price_map = fetch_ge_prices()
price_map = price_map["data"]

In [6]:
item_df = pd.DataFrame(item_map)
price_df = pd.DataFrame(price_map).T
item_df
price_df["id"] = price_df.index.astype(int)
price_df.reset_index(drop=True, inplace=True)
price_df

,high,highTime,low,lowTime,id
0,204.0,1.753109e+09,195.0,1.753109e+09,2
1,203822.0,1.753109e+09,192009.0,1.753109e+09,6
2,214000.0,1.753109e+09,194509.0,1.753109e+09,8
3,171270.0,1.753109e+09,170735.0,1.753109e+09,10
4,194116.0,1.753109e+09,191101.0,1.753109e+09,12
...,...,...,...,...,...
4160,666.0,1.753109e+09,672.0,1.753109e+09,30843
4161,831.0,1.753109e+09,830.0,1.753109e+09,30848
4162,8950000.0,1.753042e+09,8566000.0,1.753035e+09,30864
4163,270000.0,1.753108e+09,250000.0,1.753109e+09,30895


In [7]:
# merge dfs on id
full_df = pd.merge(item_df, price_df, on = "id", how = "left")
full_df = full_df[["name","id", "high", "low"]]
full_df

,name,id,high,low
0,3rd age amulet,10344,7.912208e+07,7.600000e+07
1,3rd age axe,20011,2.147484e+09,2.147484e+09
2,3rd age bow,12424,1.815000e+09,1.723130e+09
3,3rd age cloak,12437,4.983323e+08,4.780000e+08
4,3rd age druidic cloak,23345,1.215000e+09,1.181123e+09
...,...,...,...,...
4273,Zombie helmet,30321,2.356400e+05,2.087800e+05
4274,Zombie pirate key,29449,1.596800e+04,1.575000e+04
4275,Zul-andra teleport,12938,1.185300e+04,1.162700e+04
4276,Zulrah's scales,12934,2.110000e+02,2.060000e+02


In [8]:
full_df.to_parquet("../data/osrs_item_prices.parquet")